# Analyse der Kicker Manager Saison 2021/22

In diesem Notebook wird die Saison 2021/22 der Gruppe Dorf anhand von einigen grundlegenden statistischen Methoden analysiert.

Die Codepassagen lassen sich nicht ausblenden, können aber getrost ignoriert werden.
Dem interessierten Leser dienen Sie allenfalls der Nachvollziehbarkeit und Kontrolle.

## 0. Vorbereitungen

Hier werden lediglich einige technisch notwendige Vorkehrungen durchgeführt.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

Als Grundlage dient eine Tabelle mit den Punkten pro Spieltag und Manager.
Diese habe ich in eine CSV Datei übertragen.
Folgendes Kommando liest die Daten ein.

In [ ]:
df = pd.read_csv('src/season2122/data.csv')

## 1. Überblick

Die Spalten zeigen die Manager, die Reihen stehen für die einzelnen Spieltage.
Eine Zelle ist also die Punkteausbeute eines Managers an einem bestimmten Spieltag.

In [ ]:
df.head(3)

Wie in den meisten Programmiersprachen startet die Indexierung mit Null.
In unseren Daten hat das den Effekt, dass Zeile $i$ für Spieltag $i+1$ steht.
Um hier nicht immer umdenken zu müssen, gleichen wir den Index an die Spieltage an:

In [ ]:
# account for the fact that indexing usually starts with zero but we want matchdays starting with 1
df.index += 1

Jetzt passen die Indizes zu den Spieltagen:

In [ ]:
df.head(3)

In [ ]:
df.tail(4)

In [ ]:
df.cumsum()

Ein Histogramm gibt einen ersten groben Überblick über die Daten.
Es zeigt die Verteilung der Punkteausbeute innerhalb der Liga über die Saison in 5er Intervallen.

> Die Diagramme sind interaktiv, beim Überfahren mit der Maus werden weitere Details angezeigt.

In [ ]:
fig = go.Figure(data=[go.Histogram(x=df.values.flatten())])
fig.show()

## 2. Mittelwert

### 2.1 Globaler Mittelwert

In [ ]:
print(f"Gesamtdurchschnitt der Saison: {df.values.mean().round(2)}")

Wenn man also weniger als diese Punktzahl erreicht hat, hat man in der Liga unterdurchscnittlich gepunktet.

Oder anders gesagt: Erst ab dieser Punktzahl war es ein "guter" - da saisonübergreifend überdurchschnittlicher - Spieltag.

### 2.1 Mittelwert pro Manager

In [ ]:
print(df.mean())

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df.columns, y=df.mean().round(2)))
fig.add_shape(type='line', x0=-1, y0=df.values.mean(), x1=7, y1=df.values.mean(), line=dict(color='green'))
fig.show()

## 3. Extremwerte

### 3.1 Bester Spieltag Gruppe Dorf

In [ ]:
df.mean(axis=1).sort_values(ascending=False).head(5).round(2)

In [ ]:
i = df.mean(axis=1).idxmax()
df.loc[i].sort_values(ascending=False)

### 3.2 Schlechtester Spieltag Gruppe Dorf

In [ ]:
df.mean(axis=1).sort_values(ascending=True).head(5).round(2)

In [ ]:
i = df.mean(axis=1).idxmin()
df.loc[i].sort_values(ascending=False)

### 3.3 Bester Spieltag pro Manager

In [ ]:
max_per_manager = pd.concat(
    [df.idxmax().rename('matchday'), df.max().rename('points')],
    axis=1
)

# check if the max is also the max in the corresponding row ...
max_per_manager['was_md_winner'] = df.max().values == df.loc[df.idxmax().values].max(axis=1).values

print("Bester Spieltag pro Manager:")
print(f"{max_per_manager.sort_values(by='points', ascending=False)}")

Der "schlechteste beste" Spieltag:

In [ ]:
i = max_per_manager['points'].idxmin()
md = max_per_manager['matchday'].loc[i]
df.loc[md].sort_values(ascending=False)

### 3.4 Top-Werte

In [ ]:
df.where(df >= 100).count().sort_values(ascending=False)

In [ ]:
df.where(df >= 90).count().sort_values(ascending=False)

In [ ]:
df.where(df >= 80).count().sort_values(ascending=False)

In [ ]:
df.where(df >= 70).count().sort_values(ascending=False)

In [ ]:
df.where(df >= 60).count().sort_values(ascending=False)


### 3.5 Schlechtester Spieltag pro Manager

In [ ]:
min_per_manager = pd.concat(
    [df.idxmin().rename('matchday'), df.min().rename('points')],
    axis=1
)

print("Schlechtester Spieltag pro Manager")
print(f"{min_per_manager.sort_values(by='points', ascending=False)}")

### 3.6 Flop Werte

In [ ]:
df.where(df < 10).count().sort_values(ascending=False)

In [ ]:
df.where(df < 20).count().sort_values(ascending=False)

In [ ]:
df.where(df < 30).count().sort_values(ascending=False)

In [ ]:
df.where(df < 40).count().sort_values(ascending=False)

## 4. Spieltagssiege

In [ ]:
df.idxmax(axis=1).value_counts()

## 5. Liga Tabelle

### 5.1 Spitzenreiter

In [ ]:
stat_array = np.argsort(np.argsort(df.cumsum().to_numpy() * -1)) + 1
stats = pd.DataFrame(stat_array, columns=df.columns)
stats.where(stats == 1).count().sort_values(ascending=False)

### 5.2 Rote Laterne

In [ ]:
stats.where(stats == 7).count().sort_values(ascending=False)


## 6. Punkteentwicklung

In [ ]:
points_cum = df.cumsum()
points_cum.tail(3)

fig = px.line(points_cum)
fig.update_traces(mode='lines+markers')
# fig.update_yaxes(autorange="reversed")
fig.update_layout(xaxis_title="Matchday", yaxis_title="Points",)
fig.show()

## 7. Ähnliche Manager

Die Ähnlichkeit zweier Manager kann man bestimmen, indem man betrachtet, wie sehr Ihre Punkteausbeute korreliert, also voneinander abhängt.
Zwei Manager $A$ und $B$ korrelieren, wenn an Spieltagen, an denen Manager $A$ gut punktet, $B$ ebenfalls gut punktet, und an Spieltagen an denen $A$ schlecht punktet, punktet auch $B$ schlecht.
Typischerweise haben diese Manager viele gleiche Spieler, oder aber auch viele Spieler aus gleichen Vereinen (z.B. haben A und B viele "Grüne" und Freiburger ;) ).
Hätten beispielsweise zwei Manager $A$ und $B$ eine komplett identische Truppe, wäre Ihre Punkteausbeute an jedem Spieltag gleich, und somit läge der Korrelationswert bei 1.
Ist das Gegenteil der Fall, liegt dieser Wert bei 0.

Die Korrelationsmatrix zeigt uns die Zahlenwerte hierfür:

In [ ]:
df.corr()

D.h. der höchste Wert hier steht für die beiden "ähnlichsten" Manager.
Intuitiver kann man das in einer Heatmap erkennen:

In [ ]:
fig = px.imshow(df.corr())
fig.show()

In [ ]:
fig = px.scatter_matrix(df)
fig.update_layout(font=dict(size=8))
fig.show()


Korrelieren zwei Manager stark, liegen alle Punkte auf der Diagonalen - so wie das der Fall ist wenn man Manager $A$ mit sich selbst vergleicht, was quasi zwei identische Manager simuliert.
Und je "ungeordneter" die Punkte, desto weniger korrelieren zwei Manager.
Am ehesten ist hier noch ein Trend bei Didi und Pommo erkennbar, obgleich auch hier schon eine große Streuung vorhanden ist.


## 8. Standardabweichung

Einige der bisherigen Zahlen lassen bereits erahnen, dass Berthi eine sehr wankelmütige Saison erlebte.
Das kann man auch in einer einzigen Zahl ausdrücken, nämlich der Standardabweichung.
Sie kann interpretiert werden als "durchschnittliche Streuung um den Mittelwert"

Folgendes Extrembeispiel soll die Idee veranschaulichen:

Die Zahlentripel $A = (10, 40, 70)$ und $B = (39, 40, 41)$ haben beide den Mittelwert $µ=40$,
doch ist es offensichtlich, dass Streuung bei $A$ deutlich größer ist als bei $B$.
Genau das wird durch die Standardabweichung $\sigma$ ausgedrückt, indem man den Abstand zum Mittelwert misst:

$\sigma = \sqrt{\frac{\sum{(x_i-\mu)^2}}{N}}$

Hier nun die Standardabweichung pro Manager:

In [ ]:
df.std().sort_values(ascending=False)

Der Abstand zwichen Maximal- und Minimalwert ist ebenfalls schon ein guter Streuungsindikator.

In [ ]:
(df.max() - df.min()).sort_values(ascending=False)


## 9. Blick in die Zukunft ...

Hätte der 🐕 nicht  💩, hätte er die 🐈 erwischt 😃

In [ ]:
# provide data
add_mds = 4
X = np.arange(len(points_cum) + add_mds).reshape(-1, 1)
Y = points_cum.to_numpy()
D = 3
t = len(df)

# transform data to polynomial
transformer = PolynomialFeatures(degree=D)
X_poly = transformer.fit_transform(X)

# fit the linear regression model on first 34 matchdays
model = LinearRegression().fit(X_poly[:t], Y[:t])

# predict on 38 matchdays
y_pred = model.predict(X_poly)
df_pred = pd.DataFrame(y_pred, columns=df.columns)

# plot
fig = go.Figure()
colors = iter(px.colors.qualitative.Plotly)
for c in df.columns:
    color = next(colors)
    fig.add_trace(go.Scatter(x=np.arange(40), y=points_cum[c], mode='markers', name=c, line=dict(color=color)))
    fig.add_trace(go.Scatter(x=np.arange(40), y=df_pred[c], showlegend=False, line=dict(color=color)))
fig.show()

## 10. Schlusswort

Das soll es dann auch gewesen sein.
Ich hoffe der etwas andere Blick auf das Managerspiel hat euch bei lesen ähnlich Freude bereitet wie mir beim erstellen :)